# Работа №1 - Поиск ассоциативных правил

_Вариант 5. Сформировать набор из 30 транзакций на основе продуктов, употребляемых на завтрак, обед и ужин._

In [1]:
#! echo '[["Мясо", "Соус", "Вино"],["Мясо", "Соус", "Картофель", "Масло"],["Картофель", "Масло"],["Картофель", "Масло"],["Вино", "Яйца", "Бекон"],["Овощи", "Мясо", "Специи"],["Яйца", "Бекон", "Овощи", "Мясо", "Специи"],["Яйца", "Бекон", "Овощи", "Мясо", "Специи"],["Лапша", "Кетчуп"],["Пельмени", "Майонез", "Кетчуп"],["Пельмени", "Майонез", "Кетчуп"],["Овощи", "Пельмени", "Майонез", "Кетчуп"],["Овощи", "Соус", "Вино"],["Кетчуп"],["Кетчуп", "Лапша", "Пельмени", "Майонез"],["Кетчуп", "Лапша", "Пельмени", "Майонез"],["Кетчуп", "Лапша", "Мясо", "Соус", "Вино"],["Овощи", "Мясо", "Соус", "Вино"],["Майонез"],["Вино"],["Овощи", "Масло"],["Овощи", "Масло", "Картофель"],["Овощи", "Масло", "Картофель", "Мясо", "Соус"],["Овощи", "Бекон", "Яйца"],["Бекон", "Яйца"],["Яйца", "Кетчуп", "Лапша"],["Мясо"],["Мясо", "Масло"],["Мясо", "Масло", "Картофель"],["Масло", "Картофель"]]' > data/data_set.json
#! curl https://raw.githubusercontent.com/viktree/curly-octo-chainsaw/master/BreadBasket_DMS.csv > ./data/BreadBasket_DMS.csv
#! pip3 install apriori_python efficient-apriori fpgrowth-py pyarmviz pandas numpy

In [2]:
from lib import datasets
from pandas import pandas
from fpgrowth_py import fpgrowth
from efficient_apriori import apriori

In [3]:
repo_ds = datasets.from_csv("./data/BreadBasket_DMS.csv")
test_ds = datasets.from_json("./data/data_set.json")

In [4]:
_, test_rules = apriori(test_ds, min_support=0.15, min_confidence=0.6)
_, repo_rules = apriori(repo_ds, min_support=0.01, min_confidence=0.6)
repo_rules

[{Toast} -> {Coffee}]

In [5]:
_, test_rules_fpg = fpgrowth(test_ds, minSupRatio=0.15, minConf=0.6)
_, repo_rules_fpg = fpgrowth(repo_ds, minSupRatio=0.01, minConf=0.6)
repo_rules_fpg

[[{'Toast'}, {'Coffee'}, 0.7044025157232704]]

In [6]:
from lib import associative_rules
r = associative_rules.compute(test_ds)
r

{('Р’РёРЅРѕ', 'Р’РёРЅРѕ'): {'support': 0.2, 'confidence': 'INF'},
 ('Р’РёРЅРѕ', 'РњСЏСЃРѕ'): {'support': 0.1, 'confidence': 1.0},
 ('Р’РёРЅРѕ', 'РЎРѕСѓСЃ'): {'support': 0.13333333333333333, 'confidence': 2.0},
 ('РњСЏСЃРѕ', 'Р’РёРЅРѕ'): {'support': 0.1, 'confidence': 0.375},
 ('РњСЏСЃРѕ', 'РњСЏСЃРѕ'): {'support': 0.36666666666666664,
  'confidence': 'INF'},
 ('РњСЏСЃРѕ', 'РЎРѕСѓСЃ'): {'support': 0.16666666666666666,
  'confidence': 0.8333333333333334},
 ('РЎРѕСѓСЃ', 'Р’РёРЅРѕ'): {'support': 0.13333333333333333, 'confidence': 2.0},
 ('РЎРѕСѓСЃ', 'РњСЏСЃРѕ'): {'support': 0.16666666666666666, 'confidence': 5.0},
 ('РЎРѕСѓСЃ', 'РЎРѕСѓСЃ'): {'support': 0.2, 'confidence': 'INF'},
 ('РљР°СЂС‚РѕС„РµР»СЊ', 'РљР°СЂС‚РѕС„РµР»СЊ'): {'support': 0.23333333333333334,
  'confidence': 'INF'},
 ('РљР°СЂС‚РѕС„РµР»СЊ', 'РњСЏСЃРѕ'): {'support': 0.1, 'confidence': 0.75},
 ('РљР°СЂС‚РѕС„РµР»СЊ', 'РњР°СЃР»Рѕ'): {'support': 0.23333333333333334,
  'confidence': 'INF'},
 ('РљР°СЂС‚РѕС„РµР»СЊ', 'РЎРѕСѓСЃ'): {'sup